## the `summarize` module

This document reviews the `summarize` module - which takes as input a document and returns a summary of its (text) contents.

This document includes an overview of custom pipeline setup, current model set, parameters, and `.process` usage for this module.

To follow along with this demonstration be sure to initialize your krixik session with your api key and url as shown below. 

We illustrate loading these required secrets in via [python-dotenv](https://pypi.org/project/python-dotenv/), storing those secrets in a `.env` file.  This is always good practice for storing / loading secrets (e.g., doing so will reduce the chance you inadvertantly push secrets to a repo).

In [1]:
import sys 
sys.path.append('../../')
from docs.utilities.reset import reset_pipeline

In [2]:
# load secrets from a .env file using python-dotenv
from dotenv import load_dotenv
import os
load_dotenv("../.env")
MY_API_KEY = os.getenv('MY_API_KEY')
MY_API_URL = os.getenv('MY_API_URL')

# import krixik and initialize it with your personal secrets
from krixik import krixik
krixik.init(api_key = MY_API_KEY, 
            api_url = MY_API_URL)

SUCCESS: You are now authenticated.


This small function prints dictionaries very nicely in notebooks / markdown.

In [3]:
# print dictionaries / json nicely in notebooks / markdown
import json
def json_print(data):
    print(json.dumps(data, indent=2))

A table of contents for the remainder of this document is shown below.


- [pipeline setup](#pipeline-setup)
- [using the default model](#using-the-default-model)
- [recursive summarization](#recursive-summarization)
- [using a non-default model](#using-a-non-default-model)

## Pipeline setup

Below we setup a simple one module pipeline using the `summarize` module. 

In [4]:
# import custom module creation tools
from krixik.pipeline_builder.module import Module
from krixik.pipeline_builder.pipeline import CreatePipeline

# instantiate module
module_1 = Module(module_type="summarize")

# create custom pipeline object
custom = CreatePipeline(name='summarize-pipeline-1', 
                        module_chain=[module_1])

# pass the custom object to the krixik operator (note you can also do this by passing its config)
pipeline = krixik.load_pipeline(pipeline=custom)

The `summarize` module comes with a single model:

- [bart-large-cnn](https://huggingface.co/facebook/bart-large-cnn) (default)
- [text-summarization](https://huggingface.co/Falconsai/text_summarization)

These available modeling options and parameters are stored in our custom pipeline's configuration (described further in LINK HERE).  We can examine this configuration as shown below.

In [5]:
# nicely print the configuration of uor custom pipeline
json_print(custom.config)

{
  "pipeline": {
    "name": "summarize-pipeline-1",
    "modules": [
      {
        "name": "summarize",
        "models": [
          {
            "name": "bart-large-cnn"
          },
          {
            "name": "text-summarization"
          }
        ],
        "defaults": {
          "model": "bart-large-cnn"
        },
        "input": {
          "type": "text",
          "permitted_extensions": [
            ".txt",
            ".pdf",
            ".docx",
            ".pptx"
          ]
        },
        "output": {
          "type": "text"
        }
      }
    ]
  }
}


Here we can see the models and their associated parameters available for use.

In [6]:
# delete all processed datapoints belonging to this pipeline
reset_pipeline(pipeline)

## using the default model

We first define a path to a local input file.

In [7]:
# define path to an input file from examples directory
test_file = "../input_data/1984_short.txt"

Lets take a quick look at this file before processing.

In [8]:
# examine contents of input file
with open(test_file, "r") as file:
    print(file.read())

It was a bright cold day in April, and the clocks were striking thirteen.
Winston Smith, his chin nuzzled into his breast in an effort to escape the
vile wind, slipped quickly through the glass doors of Victory Mansions,
though not quickly enough to prevent a swirl of gritty dust from entering
along with him.

The hallway smelt of boiled cabbage and old rag mats. At one end of it a
coloured poster, too large for indoor display, had been tacked to the wall.
It depicted simply an enormous face, more than a metre wide: the face of a
man of about forty-five, with a heavy black moustache and ruggedly handsome
features. Winston made for the stairs. It was no use trying the lift. Even
at the best of times it was seldom working, and at present the electric
current was cut off during daylight hours. It was part of the economy drive
in preparation for Hate Week. The flat was seven flights up, and Winston,
who was thirty-nine and had a varicose ulcer above his right ankle, went
slowly, resting se

Now let's process it using our default model [bart-large-cnn](https://huggingface.co/facebook/bart-large-cnn).  Since we are using the default model we need not input the optional `modules` argument into `.process`.

In [9]:
# define path to an input file from examples directory
test_file = "../input_data/1984_short.txt"

# process a file through the pipeline
process_output = pipeline.process(local_file_path = test_file,
                                  local_save_directory=".", # save output in current directory
                                  expire_time=60*5,         # set all process data to expire in 5 minutes
                                  wait_for_process=True,    # wait for process to complete before regaining ide
                                  verbose=False)            # set verbosity to False

The output of this process is printed below.  Because the output of this particular module-model pair is a json, the process output is provided in this object is as well.  The file itself has been returned to the address noted in the `process_output_files` key.  The `file_id` of the processed file is used as a filename prefix for both output files.

In [10]:
# nicely print the output of this process
json_print(process_output)

{
  "status_code": 200,
  "pipeline": "summarize-pipeline-1",
  "request_id": "b1c10776-8c3f-4153-b6e8-941cd307bb63",
  "file_id": "402cd93b-3cf0-4196-938d-830563981992",
  "message": "SUCCESS - output fetched for file_id 402cd93b-3cf0-4196-938d-830563981992.Output saved to location(s) listed in process_output_files.",
  "warnings": [],
  "process_output": null,
  "process_output_files": [
    "./402cd93b-3cf0-4196-938d-830563981992.txt"
  ]
}


We load in the text file output from `process_output_files` below. 

In [11]:
# load in process output from file
with open(process_output['process_output_files'][0], "r") as file:
    print(file.read())  

Winston Smith walked through the glass doors of Victory Mansions. The hallway
smelt of boiled cabbage and old rag mats. At one end of
it it acoloured poster, too large for indoor display, had been tacked
to the wall. It depicted simply an enormous face, more than a
metre wide. Winston made for the stairs.

Inside the flat a fruity voice was reading out a list of
figures which had something to do with pig-iron. Winston turned a switch
and the voice sank somewhat, though the words were still distinguishable. He
moved over to the window: a smallish, frail figure, the meagreness of
his body merely emphasized by the blue overalls which were the uniform
of the party.

Winston kept his back turned to the telescreen. It was safer; though,
as he well knew, even a back can be revealing. A kilometre
away the Ministry of Truth, his place of work, towered vast and
white above the grimy landscape.

The Ministry of Truth --Minitrue, in Newspeak -- was startlingly different from
any other object in si

We have just summarized a test input of around 5500 characters into a summary of less than half that length.

But what if this is not short enough for our needs?

One of the most practical ways to achieve a shorter - and more absract but still representative - summary is to apply a summarization *recursively*. 

That is, we feed the summary created above through the summarizer again - producing a shorter summary.

We do this next.

### recursive summarization

To produce a shorter summary of our original text we feed the first summarization made above into our summarizer.

This involves executing `.process` on our pipeline in precisely the same manner we saw above.  The only difference is the file we feed into it.  Now we will feed the output summary we just created.

In [12]:
# assign the summary above to a variable for reference
first_summary = process_output['process_output_files'][0]

# process a file through the pipeline
process_output = pipeline.process(local_file_path = first_summary,
                                  local_save_directory=".", # save output in current directory
                                  expire_time=60*5,         # set all process data to expire in 5 minutes
                                  wait_for_process=True,    # wait for process to complete before regaining ide
                                  verbose=False)            # set verbosity to False

Once `.process` completes we receive our shorter summary as an output file just as when we performed summarization the first time above.

Let's examine the new shorter summary file.

In [13]:
# load in process output from file
with open(process_output['process_output_files'][0], "r") as file:
    print(file.read())  

Winston Smith walked through the glass doors of Victory Mansions. The hallway
smelled of boiled cabbage and old rag mats. At one end of
the hallway an acoloured poster, too large for indoor display, had been
tacked to the wall. It depicted simply an enormous face, more than
a metre wide.

Winston kept his back turned to the telescreen. It was safer; though,
as he well knew, even a back can be revealing. A kilometre
away the Ministry of Truth, his place of work, towered vast and
white above the grimy landscape.


This is an even shorter, more abstract, summary of the original input text.

Lets recurse one more time, achieving an even shorter summary of our original text.

Again nothing will change about how we use `.process` - we only point `local_file_path` to the output of our second summarization.

In [14]:
# assign the summary above to a variable for reference
second_summary = process_output['process_output_files'][0]

# process a file through the pipeline
process_output = pipeline.process(local_file_path = second_summary,
                                  local_save_directory=".", # save output in current directory
                                  expire_time=60*5,         # set all process data to expire in 5 minutes
                                  wait_for_process=True,    # wait for process to complete before regaining ide
                                  verbose=False)            # set verbosity to False

And we can view the now shorter summary below.

In [15]:
# load in process output from file
with open(process_output['process_output_files'][0], "r") as file:
    print(file.read())  

Winston Smith walked through the glass doors of Victory Mansions. The hallway
smelled of boiled cabbage and old rag mats. A kilometre away the
Ministry of Truth, his place of work, towered vast.


And indeed this is very terse but representative summary of our original text.

If we wanted - we could reproduce this result (of summarizing a document recursively three times) by building a new pipeline that contained three `summarize` modules in succession.

We explore just such an example in a [recursive summarization pipeline example](examples/summarize/recursive_summarize.md).

### using a non-default model

To use a non-default model we simply define it explicitly in the `modules` input to `.process`.

To use the [text-summarization](https://huggingface.co/Falconsai/text_summarization) model we pass it explicitly as 

```
modules = {
    "summarize": {
        "model": "text-summarization
    }
}
```

to `.process`.

In [16]:
# define path to an input file from examples directory
test_file = "../input_data/1984_short.txt"

# process a file through the pipeline
process_output = pipeline.process(local_file_path = test_file,
                                  local_save_directory=".", # save output in current directory
                                  expire_time=60*5,         # set all process data to expire in 5 minutes
                                  wait_for_process=True,    # wait for process to complete before regaining ide
                                  verbose=False,            # set verbosity to False
                                  modules = {
                                    "summarize": {
                                        "model": "text-summarization"
                                    }
                                })

KeyboardInterrupt: 

And we can view the summary by loading in the output file.  The offset punctuation seen in this summary is an artifact of the model, as seen in the [model card](https://huggingface.co/Falconsai/text_summarization) on huggingface.

In [ ]:
# load in process output from file
with open(process_output['process_output_files'][0], "r") as file:
    print(file.read())  

Winston Smith, his chin nuzzled into his breast in an effort to
escape the vile wind, slipped quickly through the glass doors of Victory
Mansions . At one end of it a coloured poster, too large
for indoor display, had been tacked to the wall . It was
part of the economy drive in preparation for Hate Week . The
flat was seven flights up, and Winston went slowly, resting several times
on the way .

Winston turned a switch and the voice sank somewhat, though the words
were still distinguishable . The instrument (the telescreen, it was called) could
be dimmed, but there was no way of shutting it off completely
. He moved over to the window: a smallish, frail figure, the
meagreness of his body merely emphasized by the blue overalls which were
the uniform of the party . Outside, even through the shut window-pane,
the world looked cold .

There was no way of knowing whether you were being watched at
any given moment . How often, or on what system, the Thought
Police plugged in on any individu